## CARES act college stimulus by ownership form

By Charlie Eaton, April 2, 2020

Note: Ownership data is ownership as of 2017 and may have shifted since then. Publicly traded firms may have a private equity investment indicator of "1" if there is no publicly available information about private equity exits from the investment.

In [1]:
quietly {
cd "/Users/Charlie/Desktop"
import excel using ///
"3_30_20---resort_03-24-20_estimated_higher_ed_fund_distribution_620pm.xlsx" ///
, sheet("Insts no fully online") firstrow case(lower) clear
drop if opeid==.
tostring opeid, gen(opeidstring)
replace opeidstring="0"+opeidstring if strlen(opeidstring)==6
replace opeidstring="0"+opeidstring if strlen(opeidstring)==7
replace opeidstring=substr(opeidstring,1,6)
destring opeidstring, gen(opeid6)
encode schooltype, gen(schooltype_n)
keep if schooltype_n==2
*ds, varwidth(28)
*summarize
save stimulusbyschoolandfirm, replace
}

In [2]:
quietly {
use "/Users/Charlie/Dropbox/Asymmetry by Design/Data/d_OPEID_Asymmetry.dta", clear
bysort opeid: egen maxyear=max(year)
bysort opeid: egen legalaction=max(law_enf_frst_this_yr) 
keep if maxyear==year
rename year ownershipyear
rename opeid opeid6
rename opeidstring opeidstringeaton
collapse (first) opeidstringeaton pe_ind public control_n iclevel institutionname ///
ownershipyear systemid systemnm (rawsum) legalaction claimcount, by(opeid6)
}
merge 1:1 opeid6 using stimulusbyschoolandfirm, gen(owner_stimulus_merge) keep(2 3)



. merge 1:1 opeid6 using stimulusbyschoolandfirm, gen(owner_stimulus_merge) keep
> (2 3)

    Result                           # of obs.
    -----------------------------------------
    not matched                           125
        from master                         0  (owner_stimulus_merge==1)
        from using                        125  (owner_stimulus_merge==2)

    matched                             1,688  (owner_stimulus_merge==3)
    -----------------------------------------


In [3]:
gen failgainful=0
replace failgainful=1 if regexm(gainfulemploymentrulefailed, "ail")==1
gen cashmonitor=0
replace cashmonitor=1 if regexm(hcm_dec_2019, "es")==1
quietly gen ownership="other"
quietly replace ownership="private equity" if pe_ind==1
quietly replace ownership="publicly traded" if public==1



(29 real changes made)


(24 real changes made)





In [4]:
format totaldollarsremovingonlineo pelldollarsremovingonlineon %12.0fc
quietly replace systemnm = institutionname if systemnm==""
quietly outsheet using collegestimulusschoolowner.csv, comma replace

In [5]:
table ownership, c(sum totaldollarsremovingonlineo) format(%12.0fc)
table ownership, c(sum pelldollarsremovingonlineon) format(%12.0fc)



------------------------------
      ownership | sum(total~o)
----------------+-------------
          other |  619,743,048
 private equity |  263,362,315
publicly traded |  185,934,013
------------------------------


------------------------------
      ownership | sum(pelld~n)
----------------+-------------
          other |  555,595,239
 private equity |  238,216,536
publicly traded |  163,131,542
------------------------------


In [6]:
collapse (first) opeidstringeaton pe_ind public ownership control_n iclevel institutionname ///
ownershipyear systemnm (sum) legalaction failgainful cashmonitor claimcount totaldollarsremovingonlineo pelldollarsremovingonlineon ///
, by(systemid)
replace systemnm="Paul Mitchell franchises" if systemid==454944
replace systemnm="Education Affiliates" if systemid==373784
replace systemnm="Concorde Career Colleges" if systemid==120661
rename pe_ind pe



(1 real change made)

(1 real change made)

(1 real change made)



In [7]:
gsort -totaldollarsremovingonlineo
quietly outsheet using collegestimulusfirmowner.csv, comma replace

In [8]:
quietly set linesize 200
rename claimcount borrowerdefense

In [9]:
replace legalaction=1 if legalaction!=0
list systemid systemnm ownership pe totaldollarsremovingonlineo if _n<11
list systemid systemnm borrowerdefense legalaction failgainful cashmonitor if _n<11


(9 real changes made)


      +------------------------------------------------------------------------------+
      | systemid                        systemnm         ownership   pe   totaldol~o |
      |------------------------------------------------------------------------------|
   1. |   101116                            EDMC   publicly traded    1   46,953,584 |
   2. |   242981                      EduK Group    private equity    1   37,585,485 |
   3. |   373784            Education Affiliates    private equity    1   33,450,480 |
   4. |   105215   Universal Technical Institute   publicly traded    1   27,064,170 |
   5. |   151661                         Lincoln   publicly traded    1   24,656,940 |
      |------------------------------------------------------------------------------|
   6. |   104531                     DeVry, Inc.   publicly traded    1   24,266,655 |
   7. |   454944        Paul Mitchell franchises             other    0   22,352,148 |
   8. |   212212  

In [10]:
quietly {
replace failgainful=1 if failgainful!=0
replace cashmonitor=1 if cashmonitor!=0
gen anyborrowerdefense=1 if borrowerdefense!=0
}


In [11]:
display "# of top 100 firms with predatory indicators:"
tabstat pe public failgainful cashmonitor anyborrowerdefense legalaction if _n<101, s(sum) columns(statistics) labelwidth(30)


# of top 100 firms with predatory indicators:


    variable |       sum
-------------+----------
          pe |        38
      public |        12
 failgainful |        23
 cashmonitor |        13
anyborrowe~e |        79
 legalaction |        12
------------------------


In [12]:
display "Total initial 2017 borrower defense claims against Top 100 CARES subsidy recipients"
tabstat borrowerdefense, s(sum)


Total initial 2017 borrower defense claims against Top 100 CARES subsidy recipients


    variable |       sum
-------------+----------
borrowerde~e |     12074
------------------------
